# Unit 2 Assignment: Building a Mixture of Experts (MoE) Router

**Topic:** Advanced Architecture using Groq API  
**Estimated Time:** 45-60 Minutes  
**Tools:** Python, Groq API, Dotenv

---

## 🎯 Objective
Your task is to build a **"Smart Customer Support Router"** using a Mixture of Experts (MoE) architecture.

In a real-world company, you don't want a "Generalist" AI handling everything. You want:
1.  A **Technical Expert** for bug reports.
2.  A **Billing Expert** for refund requests.
3.  A **Sales Expert** for new inquiries.

You will build a **Router** that takes a user query, decides which expert is best suited for it, and then forwards the query to that specific expert configuration.

---

## 🛠️ Requirements

### 1. Setup
- Install `groq` and `python-dotenv`.
- Set up your Groq API Key (variable: `GROQ_API_KEY`).

### 2. Define Your Experts
Create a configuration dictionary `MODEL_CONFIG` where you define your experts.
*Note: Since we are using the Groq API, we can simulate different "experts" by using different **System Prompts** while using the same base model (e.g., `mixtral-8x7b-32768`).*

- **Technical Expert:** System prompt should be rigorous, code-focused, and precise.
- **Billing Expert:** System prompt should be empathetic, financial-focused, and policy-driven.
- **General Expert:** A fallback for casual chat.

### 3. The Router (The Core Task)
Write a function `route_prompt(user_input)` that uses an LLM call to classify the intent.
- Input: User's query string.
- Output: The **Category Name** (e.g., "technical", "billing", "general").
- **Constraint:** The router must return *only* the category name, nothing else.

### 4. The Orchestrator
Write a main function `process_request(user_input)` that:
1.  Calls `route_prompt` to decide the category.
2.  Selects the correct **System Prompt** based on the category.
3.  Calls the generic LLM (Mixtral) with that specific System Prompt + User Input.
4.  Returns the final answer.

---

## 📝 Example Output

**Input:**
> "My python script is throwing an IndexError on line 5."

**Logic:**
1.  **Router** sees "Python", "IndexError" -> Classifies as **"technical"**.
2.  **Orchestrator** loads the "Technical Expert" system prompt.
3.  **Expert** responds with a code fix.

**Input:**
> "I was charged twice for my subscription this month."

**Logic:**
1.  **Router** sees "charged", "subscription" -> Classifies as **"billing"**.
2.  **Orchestrator** loads the "Billing Expert" system prompt.
3.  **Expert** responds with refund policy info.

---

## 💡 Hints
- Use `temperature=0` for the Router (you want consistency).
- Use `temperature=0.7` for the Experts (you want creativity/flexibility).
- Your routing prompt should be very clear: *"Classify this text into one of these categories: [technical, billing, general]. Return ONLY the word."*

---

## 🚀 Bonus Challenge
Can you add a **"Tool Use"** expert?
If the user asks for "current price of Bitcoin", route it to a function that (mock) fetches data, instead of just an LLM response.


In [3]:
!pip install groq langchain-groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 7.7 MB/s eta 0:00:00


In [4]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


In [10]:
# Deterministic router
router_llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0
)

# Expert model
expert_llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7
)

**Define Expert Personalities**

In [11]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a highly rigorous technical support expert. "
            "Provide precise, logical, and code-focused solutions."
        )
    },
    "billing": {
        "system_prompt": (
            "You are an empathetic billing specialist. "
            "Help with payments, refunds, subscriptions, and charges."
        )
    },
    "general": {
        "system_prompt": (
            "You are a friendly general assistant handling casual queries."
        )
    }
}

**Router Logic**

In [12]:
def route_prompt(user_input: str) -> str:

    routing_prompt = (
        "Classify the following user request into exactly one category:\n"
        "[technical, billing, general]\n\n"
        "Return ONLY the category name.\n\n"
        f"User Request: {user_input}"
    )

    response = router_llm.invoke([
        HumanMessage(content=routing_prompt)
    ])

    return response.content.strip().lower()

**Orchestrator (Expert Execution)**

In [13]:
def process_request(user_input: str) -> str:

    category = route_prompt(user_input)

    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = expert_llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_input)
    ])

    return f"[Routed to: {category.upper()}]\n\n{response.content}"

In [14]:
queries = [
    "My python code throws IndexError when accessing a list.",
    "I was charged twice for my subscription.",
    "Tell me a fun fact about space."
]

for q in queries:
    print("\nUSER:", q)
    print(process_request(q))


USER: My python code throws IndexError when accessing a list.
[Routed to: TECHNICAL]

**IndexError in Python**

An `IndexError` in Python occurs when you try to access an element in a list using an index that is out of range. This can happen when the index is either too large or too small.

**Common Causes**

1. **Out of bounds indexing**: Accessing an element at an index that is greater than or equal to the length of the list.
2. **Negative indexing**: Accessing an element at a negative index that is less than the negative length of the list.

**Example Code**
```python
my_list = [1, 2, 3]

# Out of bounds indexing
try:
    print(my_list[3])
except IndexError as e:
    print(f"Error: {e}")

# Negative indexing
try:
    print(my_list[-4])
except IndexError as e:
    print(f"Error: {e}")
```

**Solution**

To fix the `IndexError`, you can use one of the following approaches:

1. **Check the length of the list**: Before accessing an element, check if the index is within the bounds of th

**Bonus Challenge**

Can you add a **"Tool Use"** expert? If the user asks for "current price of Bitcoin", route it to a function that (mock) fetches data, instead of just an LLM response.

In [15]:
def fetch_bitcoin_price():
    # Mock response (since assignment says mock is fine)
    return "Bitcoin current price (mock): $51,240"

In [16]:
def process_request(user_input: str) -> str:

    normalized = user_input.lower()

    # -----------------------------
    # TOOL ROUTING (PRIORITY CHECK)
    # -----------------------------
    if "bitcoin" in normalized and "price" in normalized:
        tool_result = fetch_bitcoin_price()
        return f"[Routed to: TOOL EXPERT]\n\n{tool_result}"

    # -----------------------------
    # NORMAL MoE ROUTING
    # -----------------------------
    category = route_prompt(user_input)

    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = expert_llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_input)
    ])

    return f"[Routed to: {category.upper()}]\n\n{response.content}"

In [17]:
queries = [
    "What is the current price of Bitcoin?",
    "My Python list gives IndexError",
    "I was charged twice"
]

for q in queries:
    print("\nUSER:", q)
    print(process_request(q))


USER: What is the current price of Bitcoin?
[Routed to: TOOL EXPERT]

Bitcoin current price (mock): $51,240

USER: My Python list gives IndexError
[Routed to: TECHNICAL]

# Step-by-step analysis of the problem:
1. **Understanding the Error**: An `IndexError` in Python occurs when you try to access an element in a list using an index that is out of range. This means the index you're trying to access does not exist within the list.
2. **Identifying the Cause**: The cause could be due to several reasons:
   - Trying to access an index that is greater than or equal to the length of the list.
   - Trying to access a negative index that is less than -length of the list.
   - Incorrectly assuming the size of the list.

# Fixed solution:
To fix this issue, you need to ensure that the index you're trying to access is within the bounds of the list. Here's a simple example of how to safely access elements in a list:

```python
def safe_list_access(my_list, index):
    """
    Safely access an el